# Syntactic-Sugar (Book's Code)
---
At the moment, I am struggling

In [5]:
import re

In [ ]:
def desugar(code, func_name, func_args, func_body):
    """
    Replaces all occurences of 
       foo = func_name(func_args) 
    with
       func_body[x->a,y->b]
       foo = [result returned in func_body]    
    """
    # Uses Python regular expressions to simplify the search and replace,
    # see https://docs.python.org/3/library/re.html and Chapter 9 of the book

    # regular expression for capturing a list of variable names separated by commas
    arglist = ",".join([r"([a-zA-Z0-9\_\[\]]+)" for i in range(len(func_args))])
    # regular expression for capturing a statement of the form
    # "variable = func_name(arguments)"
    regexp = fr'([a-zA-Z0-9\_\[\]]+)\s*=\s*{func_name}\({arglist}\)\s*$'
    while True:
        m = re.search(regexp, code, re.MULTILINE)
        if not m: break
        newcode = func_body 
        # replace function arguments by the variables from the function invocation
        for i in range(len(func_args)): 
            newcode = newcode.replace(func_args[i], m.group(i+2))
        # Splice the new code inside
        newcode = newcode.replace('return', m.group(1) + " = ")
        code = code[:m.start()] + newcode + code[m.end()+1:]
    return code

In [12]:
def parse_func(code):
    """Parse a function definition into name, arguments and body"""
    lines = [l.strip() for l in code.split('\n')]
    regexp = r'def\s+([a-zA-Z\_0-9]+)\(([\sa-zA-Z0-9\_,]+)\)\s*:\s*'
    m = re.match(regexp, lines[0])
    return m.group(1), m.group(2).split(','), '\n'.join(lines[1:])

In [13]:
def IF(cond, a, b):
    """Version #1"""
    notcond = NAND(cond, cond)
    temp = NAND(b, notcond)    # check if condition = False
    temp1 = NAND(a, cond)      # check if condition = True
    return NAND(temp, temp1)

In [14]:
def IF(cond, a, b):
    """Version #2"""
    t1 = AND(cond, a)          # check if condition = True
    notcond = NOT(cond)
    t2 = AND(notcond, b)       # check if condition = False
    return OR(t1, t2)          # either t1 or t2 will be True, not Both

In [15]:
# Add two n-bit integers
# Use LSB first notation for simplicity

n = 5 # number of bits

def ADD(A, B):
    """
    n = fixed integer
    input = 2n bits
    output = n + 1 bits
    """
    Result = [0] * (n+1)
    Carry  = [0] * (n+1)
    Carry[0] = zero(A[0])
    
    for i in range(n):
        Result[i] = XOR(Carry[i], XOR(A[i], B[i]))
        Carry[i+1] = MAJ(Carry[i], A[i], B[i])

    Result[n] = Carry[n]
    return Result

# ADD([1, 1, 1, 0, 0], [1, 0, 0, 0, 0]) # [0, 0, 0, 1, 0, 0]

In [ ]:
def LOOKUP1(X[0], X[1], i):
    return X[0] if i == 0 else X[1]

In [ ]:
def LOOKUP2(X[0], X[1], X[2], X[3], i[0], i[1]):
    """Version #1"""
    # i[1] is the offset bit
    if i[0] == 1: # MSB = 1 -> Higher-Order Bits
        return LOOKUP1(X[2], X[3], i[1])
    else: # MSB = 0 -> Lower-Order Bits
        return LOOKUP1(X[0], X[1], i[1])

In [ ]:
def LOOKUP2(X[0], X[1], X[2], X[3], i[0], i[1]):
    """Version #2"""
    # i[1] is the offset bit
    a = LOOKUP1(X[2], X[3], i[1])
    b = LOOKUP1(X[0], X[1], i[1])
    return IF(i[0], a, b)

In [5]:
# K: number of bits
# X: array of decimal values ranging from [0, 2(K-1)]
# I: array of bits of length [0, K-1]

def LOOKUP_RECURSE(K, X, I):
    return IF(I[0],
              LOOKUP_RECURSE(I[1], X[2**(k-1)::(2**k)-1], I[1::K-1]), # I[0] is 1
              LOOKUP_RECURSE(I[1], X[::2**(k-1)-1], I[1::K-1])) # I[0] is 0
    
def L(k) -> int:
    # K = number of lines required for LOOKUP_RECURSE
    return 2*L(K-1) + 4

<em>Every finite function can be computed by a large enough Boolean Circuit</em>